# Laplacian and Gaussian Pyramids

This notebook shows how to generate pyramids of images for the purpose of image blending.  The process of generating the pyramids and blending is exeplored with two approaches:
* Using OpenCV exclusively
* Implrmenting the pyramid generation from scratch

We import the necessary libraries 

In [1]:
import cv2
import numpy as np,sys
import os
import math as mt

### Part I. Gaussian and Laplacian pyramids and blending using OpenCV

We write a function to generate the Gaussian pyramid of a picture based on the name of the file containing the image as well as the number of times the image will be reduced

In [2]:
def get_GaussPyrimid(name,times):
    """
    Function that reduces a given image certain number of times
    name: name of file containing the image
    times: number of times the image will be reduced
    returns: gaussian pyramid
    """
    # Read image from file using OpenCV
    img = cv2.imread(name)
    # Create a list of images that initially contains only the original
    # image
    gaussPy = [img]
    # We reduce the image as many times as the user has requested
    for _ in range(times):
        # Notice we updated img every time with the new reduced image...
        img = cv2.pyrDown(img)
        # ... and append to the list of images
        gaussPy.append(img)
    return gaussPy
    

Motivated by the previous function, we modify it to apply reduction to two images

In [3]:
def get_GaussPyrimids(name1,name2,size):
    """
    Function that reduces two images certain number of times
    name1, name2: names of files containing the images
    times: number of times the image will be reduced
    returns: gaussian pyramids of images with names name1 and name2
    """
    # Read images from corresponding files
    img1 = cv2.imread(name1); img2 = cv2.imread(name2)
    # Initialize two lists with corresponding images
    gaussPy1 = [img1]; gaussPy2 = [img2]
    # We reduce the image as many times as the user has requested ... 
    for _ in range(size):
        # ... and append to the lists, respectively
        img1 = cv2.pyrDown(img1); gaussPy1.append(img1)
        img2 = cv2.pyrDown(img2); gaussPy2.append(img2)
    return gaussPy1,gaussPy2

We now create the function that will generate the Laplacian pyramids of two given images

In [4]:
def get_LaplacePyramid(name1,name2,size):
    """
    Function that creates Laplacian pyramids of two images
    name1, name2: names of files containing the images
    size: number of levels in the lapalacian pyramid
    returns: laplacian pyramids of images with names name1 and name2
    """
    # We first get the gaussian pyramids of each image. Notice 
    # the function written before is invoked here
    gaussPy1,gaussPy2 = get_GaussPyrimids(name1,name2,size)
    # We create a list of Laplacian pyramids; we initialize each list 
    # with the deeper level (smallest image) of each Gaussian pyramid
    Lapl1 = [gaussPy1[size]]; Lapl2 = [gaussPy2[size]]
    # We loop over each element of the Gaussian pyramid. Notice the 
    # looping begins with the smallest image in the Gaussian pyramid 
    # (deepest level)
    # For each element of the Gaussian pyramid...
    for k in range(size,0,-1):
        # Increase size of images in turn...
        G1 = cv2.pyrUp(gaussPy1[k]); G2 = cv2.pyrUp(gaussPy2[k])
        #print G1.shape, G2.shape
        # ... take respective differences with images in turn ...
        L1 = cv2.subtract(gaussPy1[k-1],G1); L2 = cv2.subtract(gaussPy2[k-1],G2)
        # ... and append to the list of Laplacian pyramids
        #print k , L1.shape, L2.shape
        Lapl1.append(L1); Lapl2.append(L2)
    return Lapl1,Lapl2

We are now ready to genearate the blending of images

In [5]:
def blendImages(name1,name2,name_blend,size):
    """
    This function genearated the blending of two images.
    The blending takes place the middle part of each image
    name1, name2: files' names containing images
    name_blend: name of file containing the blended image
    size: size of pyramids (number of levels in pyramids) generated
          to do the blending
    returns: None but writes the resulting blended image to file
    """
    # Get Laplacian pyramids
    lpA,lpB = get_LaplacePyramid(name1,name2,size)
    # We intialize an empty list of images
    list_img = []
    # For each tuple in the cartesian product of Laplacian pyramids...
    for la,lb in zip(lpA,lpB):
         #row,col,dpt = la.shape
        # We get dimension of element of Laplacian pyramid in turn ...
        row,col = la.shape[:2]
        # ... grab the left and right halves of corresponding elements...
        left_img = la[:,0:col/2]; right_img = lb[:,col/2:]
        # ... stack them horizontally to do sewing ...
        ls = np.hstack((left_img,right_img))
        # ... append the recently sewed image
        list_img.append(ls)
    # We get the first element of sewed images... 
    ls_ = list_img[0]
    # ... and for the remaining elements in chaing of sewed images...
    for item in list_img[1:]:
        # ... increase the size ...
        ls_ = cv2.pyrUp(ls_)
        # ... and add it to the element in turn..
        ls_ = cv2.add(ls_,item)
    
    # Write image to file
    cv2.imwrite(name_blend,ls_)

We finally write a function that generates a sequence of blended images

In [6]:
def makeSequenceBlends(name1,name2,max_size):
    """
    Function that generates a sequence of blended images
    name1, name2: names of files containing the images to be blended
    max_size: maximum size of pyramids genereted for blending
    returns: None but writes sequence of images to files 
    """
    # For all levels of blending ...
    for size in range(max_size):
        # Generate name of file containing blended image in turn
        output_name = 'blend_0'+ str(size) +'.jpg'
        # Show name of file on screen...
        print output_name
        # ... and perform the blending itself
        blendImages(name1,name2,output_name,size)

In [7]:
makeSequenceBlends('apple.jpg','orange.jpg',8)

blend_00.jpg
blend_01.jpg
blend_02.jpg
blend_03.jpg
blend_04.jpg
blend_05.jpg
blend_06.jpg
blend_07.jpg


### Part II. Gaussian and Laplacian pyramids and blending from scratch

We now proceed to repeat the process followed above, this time by implementing the function without using OpenCV

#### A. Gaussian pyramids. The reduce function

We first, for testing purposes, write the kernel function and test it with a simple example

In [8]:
a_ = 0.4
wHat = np.array([0.25 - (a_/2),0.25,a_,0.25,0.25-(a_/2)])
print wHat

[ 0.05  0.25  0.4   0.25  0.05]


In [9]:
def convPyrA(xTest,wHat):
    """
    Function that performs convolution of a vector with a separable matrix
    xTest: input vector to be convoluted
    wHat: kernel used for convolution
    returns: reduced vector resulting from convolution
    """
    
    # We find the length of resulting convoluted vector according to the
    # lenght of input vector
    if len(xTest)%2 == 0:
        len_ = len(xTest)//2
    else:
        len_ = (len(xTest)//2) + 1
    
    # Set list of zeros with dimension of the resulting vector
    xReduced = [0]*len_
    
    # We first deal with all the entries in the resulting vector that 
    # are NOT the edges
    for i in range(1,len_-1):
        # Subset the input vector according the index of the convoluted
        # vector to be computed and store it in aux_
        # Keep in mind that:
        # a. The ith index of the resulting vector corresponds
        #    to the index 2i of the original vector
        # b. The ith position of resulting vector is the center and
        #    we need to collect two elements before and after the center
        #    This is [i-2,i-1,i,i+1,i+2]
        # c. In order to grab the last element we need to add 1 to the last
        #    index. This is why in the following line we have 3 = 2 + 1
        aux_ = np.array(xTest[2*i - 2 : 2*i + 3])
        # Peform the inner product of kernel with the recently obtained
        # aux_ vector to find the convolution
        xReduced[i] = np.dot(aux_,wHat)
    # We now need to take care of the edges
    # For the first cell (index zero in Python)
    # As before, we need to add 1 at the last index to ensure we are 
    # getting the last element of array: 3 = 2 + 1
    aux_ = np.array(xTest[:3])
    # For the edges the kernel has to be reduced in order to have overlap
    # with the vector; we collect the "right-hand side" of kernel
    # and store it auxiliary variable
    wAux = np.array(wHat[2:])
    # Perform inner product for convultion and correct result 
    # to ensure our reduced kernel was normalized
    xReduced[0] = np.dot(wAux,aux_)/sum(wAux)
    # We proceed in a similar fashion for last index
    aux_ = np.array(xTest[len(xTest)-3:])
    # This time the kernel is same as before but in opposite order, [::-1]
    wAux = wAux[::-1]
    # As before, we compute inner product for convolution and normalize
    # kernel
    xReduced[len_-1] = np.dot(wAux,aux_)/sum(wAux)
    
    return np.array(xReduced)
    

Let's test the recently created function with a simple vector

In [10]:
xTest = np.array([6,8,1,5,1,9,5,7,9])
print xTest
print convPyrA(xTest,wHat)

[6 8 1 5 1 9 5 7 9]
[ 6.35714286  4.          4.2         6.5         8.        ]


Motivated by the previous function we now extend the idea to apply it to a matrix. Here is where the separability of the kernel helps; we will convolute rows first and then columns

In [11]:
def convPyrMatrix(A,wHat):
    """
    This function extends the capabilities of convPyrA(xTest,wHat) to
    make it useful for matrices 
    A: image matrix to be reduced
    wHat: kernel to perform the convolution
    returns: matrix representing the reduced image
    NOTE. This function assumes the matrix A is two-dimensional
    """
    # We get number of rows and columns of given matrix A ...
    rowsA,colsA = A.shape[:2]
    # ... and set a matrix of zeros with the same shape
    Afinal = np.zeros((rowsA,colsA))

    # WE FIRST CONVOLUTE ROWS:
    for k in range(rowsA):
        # Get the row in turn and store temporarily ...
        vec_ =  np.array(A[k,:])
        # ... to convolute with kernel. Store result in temp. variable
        aux_ = convPyrA(vec_,wHat)
        # Determine length of resulting convolution and store 
        # result in corresponding section of output matrix
        rowsAux =  aux_.shape[0]
        Afinal[k,:rowsAux] = aux_
        
    #lenA = Afinal.shape[1]
    # WE NOW CONVOLUTE COLUMNS:
    for k in range(rowsAux):
        # We proceed in a similar fashion as before, this time 
        # using columns of the resulting matrix Afinal
        vec_ = np.array(Afinal[:,k])
        aux_ = convPyrA(vec_,wHat)
        colsAux =  aux_.shape[0]
        Afinal[:colsAux,k] = aux_
            
    return Afinal[:rowsAux,:colsAux]

The previous function, as mentioned in the documentation, is designed only for two-dimensional 
matrices (no color index is considered). We build upon it to extend it to its three-dimensional version where color is included

In [12]:
def GaussPyr3D(A,a_):
    """
    This function uses the previosuly implemented function, convPyrMatrix(A,wHat),
    to apply it to a three-dimensional matrix as the ones provided by OpenCV.
    Remember that the third index in matrix A is related with BGR of the pixel (x,y)
    A: three-dimensional matrix representation of an image
    a_: Parameter that uniquely defines the kernel
    returns: three-dimensional matrix representing reduced image
    """
    # The kernel is defined
    wHat = np.array([0.25 - (a_/2),0.25,a_,0.25,0.25-(a_/2)])
    #aux = aApple[:,:,0]
    
    # Apply the "two-dimensional" function to the "B" color, index 0
    aux = convPyrMatrix(A[:,:,0],wHat)
    # Get shape of resulting matrix and set a three-dimensional
    # matrix of zeros accordingly
    rows,cols = aux.shape
    Afin = np.zeros((rows,cols,3))
    # Store result in the corresponding "B" color of reduced matrix
    Afin[:,:,0] = aux
    # Repeat the same process but this time for "G" and "R" color indices
    for k in range(1,3):
        aux = convPyrMatrix(A[:,:,k],wHat)
        Afin[:,:,k] = aux
    return Afin

We now use the function GaussPyr3D(A,a_) to make the reduction of a given image

In [13]:
def reduceImage(name_input,name_output):
    """
    Function that takes an image and reduces it by colvolution
    name_input: name if image to be reduced
    name_output: name of resulting image
    returns: None, but saves to file the resulting image
    """
    # Set "customary" value of parameter for kernel
    a_ = 0.4
    # Load matrix corresponding to original image
    Ainput = cv2.imread(name_input)
    # Apply the reduce function to matrix...
    Aoutput = GaussPyr3D(Ainput,a_)
    # ... and save it to file with chose name
    cv2.imwrite(name_output,Aoutput)

Finally, we generate a Gaussian pyramid of given size

In [14]:
def reduceImageSequence(name_input,size):
    """
    This function generates a sequence of reduced images
    name_input: name of original image
    size: size of Gaussian pyramid; number of times the image will be reduced
    returns: none, but writes each image of sequence to file
    """
    # Set value of parameter for kernel
    a_ = 0.4
    # Load matrix corresponding to image
    Aimg = cv2.imread(name_input)
    # For every level in pyramid...
    for k in range(size):
        # Set the name of file...
        name_output = 'reduced_0' + str(k) + '.jpg' 
        # and write to file
        cv2.imwrite(name_output,Aimg)
        print "Image " , name_output , "with size ", Aimg.shape[:2] ,"has been created"
        # reduce matrix
        Aimg = GaussPyr3D(Aimg,a_)

In [15]:
reduceImageSequence('apple.jpg',5)

Image  reduced_00.jpg with size  (512, 512) has been created
Image  reduced_01.jpg with size  (256, 256) has been created
Image  reduced_02.jpg with size  (128, 128) has been created
Image  reduced_03.jpg with size  (64, 64) has been created
Image  reduced_04.jpg with size  (32, 32) has been created
